In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import json

In [3]:
# Spark Constants
APP_NAME = 'assignment1'
MASTER = 'local[*]'

# Input Constants
INPUT_METADATA_FILE = 'tracks.csv'
INPUT_FEATURES_FILE = 'features.csv'

In [4]:
conf = SparkConf().setAppName(APP_NAME).setMaster(MASTER)
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.appName(APP_NAME).master(MASTER).getOrCreate()

23/04/13 09:17:14 WARN Utils: Your hostname, pedro-duarte resolves to a loopback address: 127.0.1.1; using 192.168.32.217 instead (on interface wlp2s0)
23/04/13 09:17:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 09:17:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [91]:
def try_decoding(content: str):
    try:
        return json.loads(content) if content is not None else None
    except json.JSONDecodeError as e:
        return content

In [92]:
data = spark.read.csv(INPUT_METADATA_FILE, quote='"', escape='"', multiLine=True, inferSchema=True).rdd
headerRow = ['_'.join(header) if idx > 0 else 'track_id' for idx, header in enumerate(zip(*[list(line.asDict().values()) for line in data.take(2)]))]

headerRows = data.take(3)
data = data.filter(lambda row: row not in headerRows)

firstRow = data.first()

data = data \
  .map(lambda row: list(row.asDict().values())) \
  .map(lambda row: {k: try_decoding(v) for k, v in zip(headerRow, row)}) \
  .map(json.dumps)

ds = spark.read.json(data)
ds.schema

StructType([StructField('album_comments', LongType(), True), StructField('album_date_created', StringType(), True), StructField('album_date_released', StringType(), True), StructField('album_engineer', StringType(), True), StructField('album_favorites', LongType(), True), StructField('album_id', LongType(), True), StructField('album_information', StringType(), True), StructField('album_listens', LongType(), True), StructField('album_producer', StringType(), True), StructField('album_tags', StringType(), True), StructField('album_title', StringType(), True), StructField('album_tracks', LongType(), True), StructField('album_type', StringType(), True), StructField('artist_active_year_begin', StringType(), True), StructField('artist_active_year_end', StringType(), True), StructField('artist_associated_labels', StringType(), True), StructField('artist_bio', StringType(), True), StructField('artist_comments', LongType(), True), StructField('artist_date_created', StringType(), True), StructFi

In [93]:
data = ds.rdd.filter(lambda v: v['set_subset'] == 'small')
data.count() # 8000 lines

8000